In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import os

# Đọc dữ liệu
print("Đang đọc dữ liệu...")
csv_files = [f for f in os.listdir("weather_archive") if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("Không tìm thấy file CSV trong thư mục weather_archive")

data_file = os.path.join("weather_archive", csv_files[0])
df = pd.read_csv(data_file)

# Kiểm tra và làm sạch dữ liệu
print("Đang xử lý dữ liệu...")
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna(subset=['Temp_Max', 'Temp_Min', 'Weather_Code'])
df['Weather_Code'] = df['Weather_Code'].astype(int)

# Định nghĩa các mã thời tiết hợp lệ và mô tả
weather_code_descriptions = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Drizzle: Light intensity",
    53: "Drizzle: Moderate intensity",
    55: "Drizzle: Dense intensity",
    56: "Freezing drizzle: Light",
    57: "Freezing drizzle: Dense",
    61: "Rain: Slight",
    63: "Rain: Moderate",
    65: "Rain: Heavy",
    66: "Freezing rain: Light",
    67: "Freezing rain: Heavy",
    71: "Snow fall: Slight",
    73: "Snow fall: Moderate",
    75: "Snow fall: Heavy",
    77: "Snow grains",
    80: "Rain showers: Slight",
    81: "Rain showers: Moderate",
    82: "Rain showers: Violent",
    85: "Snow showers: Slight",
    86: "Snow showers: Heavy",
    95: "Thunderstorm: Slight or moderate",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

# Thêm các đặc trưng thời gian
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfYear'] = df['Date'].dt.dayofyear
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Mã hóa tỉnh thành
le_province = LabelEncoder()
df['Province_Code'] = le_province.fit_transform(df['Province'])

# Tạo thư mục lưu kết quả dự đoán
os.makedirs('xgboost_predictions', exist_ok=True)

# Dự đoán 7 ngày tiếp theo
last_date = df['Date'].max()
future_dates = [last_date + timedelta(days=i+1) for i in range(7)]

# Tạo DataFrame để lưu kết quả dự đoán
prediction_results = pd.DataFrame()

# Lấy danh sách các tỉnh thành
provinces = df['Province'].unique()

# Tạo DataFrame để lưu đánh giá mô hình
model_metrics = pd.DataFrame(columns=['Province', 'RMSE_Temp_Max', 'RMSE_Temp_Min', 'MAE_Weather_Code'])

print(f"Dự báo thời tiết cho {len(provinces)} tỉnh thành...")

for province_name in provinces:
    print(f"Đang xử lý cho tỉnh: {province_name}")
    
    # Lọc dữ liệu theo tỉnh
    province_data = df[df['Province'] == province_name].sort_values('Date')
    
    # Kiểm tra xem có đủ dữ liệu không
    if len(province_data) < 14:  # Cần ít nhất 14 ngày để có đủ dữ liệu huấn luyện
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu")
        continue
    
    # Tạo các đặc trưng với độ trễ (lag features)
    # Tạo lag features cho nhiệt độ cao, thấp và mã thời tiết
    for lag in range(1, 8):  # Tạo lag từ 1 đến 7 ngày
        province_data[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].shift(lag)
        province_data[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].shift(lag)
        province_data[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].shift(lag)
    
    # Tạo đặc trưng trung bình động (rolling mean)
    for window in [3, 5, 7]:
        province_data[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].rolling(window=window).mean()
        province_data[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].rolling(window=window).mean()
        province_data[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].rolling(window=window).mean()
    
    # Loại bỏ các hàng có giá trị NaN sau khi tạo đặc trưng
    province_data = province_data.dropna()
    
    if len(province_data) < 10:  # Kiểm tra lại sau khi đã tạo đặc trưng
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu sau khi tạo đặc trưng")
        continue
    
    # Chia tập dữ liệu
    train_data = province_data.copy()
    
    # Tạo các đặc trưng cho dự đoán
    feature_cols = [col for col in train_data.columns if col.startswith(('Temp_Max_Lag', 'Temp_Min_Lag', 
                                                                        'Weather_Code_Lag', 'Temp_Max_Roll', 
                                                                        'Temp_Min_Roll', 'Weather_Code_Roll'))]
    feature_cols += ['Month', 'Day', 'DayOfYear', 'DayOfWeek']
    
    X_train = train_data[feature_cols]
    y_temp_max = train_data['Temp_Max']
    y_temp_min = train_data['Temp_Min']
    y_weather_code = train_data['Weather_Code']
    
    # Kiểm tra mã thời tiết có trong dữ liệu của tỉnh này
    unique_weather_codes = y_weather_code.unique()
    print(f"  Mã thời tiết trong dữ liệu của {province_name}: {unique_weather_codes}")
    
    # Huấn luyện mô hình XGBoost cho nhiệt độ cao
    model_temp_max = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )
    model_temp_max.fit(X_train, y_temp_max)
    
    # Huấn luyện mô hình XGBoost cho nhiệt độ thấp
    model_temp_min = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )
    model_temp_min.fit(X_train, y_temp_min)
    
    # Huấn luyện mô hình XGBoost cho mã thời tiết (cũng sử dụng hồi quy)
    model_weather_code = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )
    model_weather_code.fit(X_train, y_weather_code)
    
    # Đánh giá mô hình trên tập huấn luyện
    train_pred_temp_max = model_temp_max.predict(X_train)
    train_pred_temp_min = model_temp_min.predict(X_train)
    train_pred_weather_code_raw = model_weather_code.predict(X_train)
    
    # Tìm mã thời tiết gần nhất cho các dự đoán
    def find_nearest_weather_code(pred_code, valid_codes=None):
        if valid_codes is None:
            valid_codes = list(weather_code_descriptions.keys())
        
        valid_codes = np.array(valid_codes)
        idx = (np.abs(valid_codes - pred_code)).argmin()
        return valid_codes[idx]
    
    # Áp dụng hàm để tìm mã thời tiết hợp lệ
    train_pred_weather_code = np.array([find_nearest_weather_code(code, unique_weather_codes) 
                                        for code in train_pred_weather_code_raw])
    
    # Tính RMSE cho nhiệt độ và MAE cho mã thời tiết
    rmse_temp_max = np.sqrt(mean_squared_error(y_temp_max, train_pred_temp_max))
    rmse_temp_min = np.sqrt(mean_squared_error(y_temp_min, train_pred_temp_min))
    mae_weather_code = mean_absolute_error(y_weather_code, train_pred_weather_code)
    
    # Lưu metrics
    model_metrics = pd.concat([model_metrics, pd.DataFrame({
        'Province': [province_name],
        'RMSE_Temp_Max': [rmse_temp_max],
        'RMSE_Temp_Min': [rmse_temp_min],
        'MAE_Weather_Code': [mae_weather_code]
    })])
    
    # Dự đoán cho 7 ngày tiếp theo
    future_predictions = []
    
    # Lấy dữ liệu gần nhất để dự đoán ngày đầu tiên
    last_row = province_data.iloc[-1].copy()
    
    for i in range(7):
        # Chuẩn bị dữ liệu cho ngày dự đoán
        future_date = future_dates[i]
        predict_row = pd.Series(dtype='float64')
        
        # Thêm đặc trưng thời gian
        predict_row['Month'] = future_date.month
        predict_row['Day'] = future_date.day
        predict_row['DayOfYear'] = future_date.dayofyear
        predict_row['DayOfWeek'] = future_date.dayofweek
        
        # Cập nhật lag features dựa trên dự đoán trước đó
        if i == 0:
            # Ngày đầu tiên: sử dụng dữ liệu thực tế
            for lag in range(1, 8):
                if lag <= len(province_data):
                    predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].iloc[-lag]
                    predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].iloc[-lag]
                    predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].iloc[-lag]
                else:
                    # Nếu không đủ dữ liệu, sử dụng giá trị trung bình
                    predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].mean()
                    predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].mean()
                    predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].mean()
            
            # Tính rolling mean dựa trên dữ liệu thực tế
            for window in [3, 5, 7]:
                if window <= len(province_data):
                    predict_row[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].iloc[-window:].mean()
                    predict_row[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].iloc[-window:].mean()
                    predict_row[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].iloc[-window:].mean()
                else:
                    predict_row[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].mean()
                    predict_row[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].mean()
                    predict_row[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].mean()
        else:
            # Từ ngày thứ 2 trở đi: cập nhật lag từ dự đoán trước đó
            for lag in range(1, 8):
                if lag <= i:
                    # Lấy từ dự đoán trước đó
                    predict_row[f'Temp_Max_Lag{lag}'] = future_predictions[i-lag]['Temp_Max']
                    predict_row[f'Temp_Min_Lag{lag}'] = future_predictions[i-lag]['Temp_Min']
                    predict_row[f'Weather_Code_Lag{lag}'] = future_predictions[i-lag]['Weather_Code']
                else:
                    # Lấy từ dữ liệu thực tế
                    idx = lag - i - 1
                    if idx < len(province_data):
                        predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].iloc[-(lag-i)]
                        predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].iloc[-(lag-i)]
                        predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].iloc[-(lag-i)]
                    else:
                        predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].mean()
                        predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].mean()
                        predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].mean()
            
            # Cập nhật rolling mean từ dự đoán trước đó
            for window in [3, 5, 7]:
                if window <= i + 1:
                    # Có đủ dự đoán trước đó + dữ liệu thực tế
                    temp_max_vals = []
                    temp_min_vals = []
                    weather_code_vals = []
                    
                    for w in range(window):
                        if w < i:
                            temp_max_vals.append(future_predictions[i-w-1]['Temp_Max'])
                            temp_min_vals.append(future_predictions[i-w-1]['Temp_Min'])
                            weather_code_vals.append(future_predictions[i-w-1]['Weather_Code'])
                        else:
                            idx = w - i
                            if idx < len(province_data):
                                temp_max_vals.append(province_data['Temp_Max'].iloc[-idx-1])
                                temp_min_vals.append(province_data['Temp_Min'].iloc[-idx-1])
                                weather_code_vals.append(province_data['Weather_Code'].iloc[-idx-1])
                            else:
                                temp_max_vals.append(province_data['Temp_Max'].mean())
                                temp_min_vals.append(province_data['Temp_Min'].mean())
                                weather_code_vals.append(province_data['Weather_Code'].mean())
                    
                    predict_row[f'Temp_Max_Roll{window}'] = np.mean(temp_max_vals)
                    predict_row[f'Temp_Min_Roll{window}'] = np.mean(temp_min_vals)
                    predict_row[f'Weather_Code_Roll{window}'] = np.mean(weather_code_vals)
                else:
                    # Không đủ dữ liệu, sử dụng trung bình
                    predict_row[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].mean()
                    predict_row[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].mean()
                    predict_row[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].mean()
        
        # Chuyển đổi Series thành DataFrame cho dự đoán
        predict_df = pd.DataFrame([predict_row])
        
        # Đảm bảo tất cả các đặc trưng đều có
        for col in feature_cols:
            if col not in predict_df.columns:
                predict_df[col] = 0
        
        # Chỉ giữ lại các đặc trưng trong mô hình
        predict_df = predict_df[feature_cols]
        
        # Dự đoán
        temp_max_pred = model_temp_max.predict(predict_df)[0]
        temp_min_pred = model_temp_min.predict(predict_df)[0]
        weather_code_pred_raw = model_weather_code.predict(predict_df)[0]
        
        # Chuyển đổi dự đoán mã thời tiết về mã thực tế gần nhất
        weather_code_pred = find_nearest_weather_code(weather_code_pred_raw, unique_weather_codes)
        
        # Lấy mô tả thời tiết dựa trên mã
        weather_description = weather_code_descriptions.get(weather_code_pred, "Unknown")
        
        # Thêm vào danh sách dự đoán
        future_predictions.append({
            'Date': future_date,
            'Temp_Max': temp_max_pred,
            'Temp_Min': temp_min_pred,
            'Weather_Code': weather_code_pred,
            'Weather_Description': weather_description
        })
    
    # Chuyển danh sách dự đoán thành DataFrame
    province_predictions = pd.DataFrame(future_predictions)
    province_predictions['Province'] = province_name
    
    # Thêm vào kết quả tổng thể
    prediction_results = pd.concat([prediction_results, province_predictions])
    
    # Vẽ biểu đồ dự đoán
    plt.figure(figsize=(15, 10))
    
    # Plot cho nhiệt độ
    plt.subplot(2, 1, 1)
    plt.plot(province_data['Date'][-14:], province_data['Temp_Max'][-14:], 'r-', marker='o', label='Nhiệt độ cao thực tế')
    plt.plot(province_data['Date'][-14:], province_data['Temp_Min'][-14:], 'b-', marker='o', label='Nhiệt độ thấp thực tế')
    plt.plot(province_predictions['Date'], province_predictions['Temp_Max'], 'r--', marker='x', label='Nhiệt độ cao dự đoán')
    plt.plot(province_predictions['Date'], province_predictions['Temp_Min'], 'b--', marker='x', label='Nhiệt độ thấp dự đoán')
    plt.title(f'Dự báo nhiệt độ 7 ngày cho {province_name}')
    plt.xlabel('Ngày')
    plt.ylabel('Nhiệt độ (°C)')
    plt.grid(True)
    plt.legend()
    
    # Plot cho mã thời tiết
    plt.subplot(2, 1, 2)
    plt.plot(province_data['Date'][-14:], province_data['Weather_Code'][-14:], 'g-', marker='o', label='Mã thời tiết thực tế')
    plt.plot(province_predictions['Date'], province_predictions['Weather_Code'], 'g--', marker='x', label='Mã thời tiết dự đoán')
    
    # Thêm nhãn mã thời tiết
    for i, date in enumerate(province_data['Date'][-14:]):
        code = province_data['Weather_Code'].iloc[-14+i]
        desc = weather_code_descriptions.get(code, "Unknown")
        plt.annotate(f"{code}", (date, code), textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
    
    for i, date in enumerate(province_predictions['Date']):
        code = province_predictions['Weather_Code'].iloc[i]
        desc = province_predictions['Weather_Description'].iloc[i]
        plt.annotate(f"{code}", (date, code), textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
    
    plt.title(f'Dự báo mã thời tiết 7 ngày cho {province_name}')
    plt.xlabel('Ngày')
    plt.ylabel('Mã thời tiết')
    plt.grid(True)
    plt.legend()
    
    # Thêm subplot cho mô tả thời tiết
    plt.figtext(0.5, 0.01, "Dự báo 7 ngày tới", ha='center', fontsize=12, fontweight='bold')
    for i, pred in enumerate(future_predictions):
        y_pos = 0.08 - i * 0.01
        if y_pos < 0:
            break
        plt.figtext(0.5, y_pos, 
                  f"{pred['Date'].strftime('%d/%m/%Y')}: {pred['Weather_Description']} ({pred['Weather_Code']})", 
                  ha='center', fontsize=8)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)  # Để có không gian cho mô tả
    plt.savefig(f'xgboost_predictions/{province_name}_forecast.png')
    plt.close()
    
    print(f"  ✓ Đã dự báo cho tỉnh {province_name}")

# Xếp lại và lưu kết quả dự đoán
prediction_results = prediction_results[['Province', 'Date', 'Temp_Max', 'Temp_Min', 'Weather_Code', 'Weather_Description']]
prediction_results.to_csv('xgboost_predictions/weather_forecast_7days.csv', index=False)

# Lưu metrics đánh giá mô hình
model_metrics.to_csv('xgboost_predictions/model_metrics.csv', index=False)

# Tạo báo cáo tổng hợp
summary_file = 'xgboost_predictions/summary_report.txt'
with open(summary_file, 'w', encoding='utf-8') as f:
    f.write("BÁO CÁO DỰ BÁO THỜI TIẾT BẰNG XGBOOST\n")
    f.write("=====================================\n\n")
    
    f.write(f"Thời gian tạo báo cáo: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Số tỉnh thành được dự báo: {len(provinces)}\n")
    f.write(f"Khoảng thời gian dự báo: {future_dates[0].strftime('%Y-%m-%d')} đến {future_dates[-1].strftime('%Y-%m-%d')}\n\n")
    
    f.write("ĐÁNH GIÁ HIỆU SUẤT MÔ HÌNH\n")
    f.write("=========================\n")
    f.write(f"RMSE trung bình Nhiệt độ cao: {model_metrics['RMSE_Temp_Max'].mean():.4f}\n")
    f.write(f"RMSE trung bình Nhiệt độ thấp: {model_metrics['RMSE_Temp_Min'].mean():.4f}\n")
    f.write(f"MAE trung bình Mã thời tiết: {model_metrics['MAE_Weather_Code'].mean():.4f}\n\n")
    
    f.write("BẢNG MÃ THỜI TIẾT\n")
    f.write("================\n")
    for code, description in weather_code_descriptions.items():
        f.write(f"{code}: {description}\n")
    
    f.write("\nTHÔNG TIN BỔ SUNG\n")
    f.write("===============\n")
    f.write("Mô hình XGBoost với các đặc trưng:\n")
    f.write("- Giá trị lag từ 1-7 ngày\n")
    f.write("- Trung bình động 3, 5, 7 ngày\n")
    f.write("- Các đặc trưng thời gian (tháng, ngày, ngày trong năm, ngày trong tuần)\n\n")
    
    f.write("Tham số mô hình XGBoost:\n")
    f.write("- n_estimators: 100\n")
    f.write("- learning_rate: 0.1\n")
    f.write("- max_depth: 5\n")
    f.write("- subsample: 0.8\n")
    f.write("- colsample_bytree: 0.8\n")

print(f"\nĐã hoàn thành dự báo thời tiết cho {len(provinces)} tỉnh thành!")
print(f"Kết quả được lưu trong thư mục 'xgboost_predictions'")
print(f"File dự báo: 'xgboost_predictions/weather_forecast_7days.csv'")
print(f"Báo cáo tổng hợp: '{summary_file}'")

Đang đọc dữ liệu...
Đang xử lý dữ liệu...
Dự báo thời tiết cho 63 tỉnh thành...
Đang xử lý cho tỉnh: An Giang
  Mã thời tiết trong dữ liệu của An Giang: [63 61 55 51 65  3 53]
  ✓ Đã dự báo cho tỉnh An Giang
Đang xử lý cho tỉnh: Bà Rịa - Vũng Tàu
  Mã thời tiết trong dữ liệu của Bà Rịa - Vũng Tàu: [51  3 53 55 63]
  ✓ Đã dự báo cho tỉnh Bà Rịa - Vũng Tàu
Đang xử lý cho tỉnh: Bạc Liêu
  Mã thời tiết trong dữ liệu của Bạc Liêu: [63 55 51 65  3 53 61]
  ✓ Đã dự báo cho tỉnh Bạc Liêu
Đang xử lý cho tỉnh: Bắc Giang
  Mã thời tiết trong dữ liệu của Bắc Giang: [61  3 51 53 65 63  0]
  ✓ Đã dự báo cho tỉnh Bắc Giang
Đang xử lý cho tỉnh: Bắc Kạn
  Mã thời tiết trong dữ liệu của Bắc Kạn: [51  3 53 65 61 63]
  ✓ Đã dự báo cho tỉnh Bắc Kạn
Đang xử lý cho tỉnh: Bắc Ninh
  Mã thời tiết trong dữ liệu của Bắc Ninh: [61  3 51 65 63 55  1]
  ✓ Đã dự báo cho tỉnh Bắc Ninh
Đang xử lý cho tỉnh: Bến Tre
  Mã thời tiết trong dữ liệu của Bến Tre: [51 55 53  3 61 63]
  ✓ Đã dự báo cho tỉnh Bến Tre
Đang xử lý c

In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
import os

# Đọc dữ liệu
print("Đang đọc dữ liệu...")
csv_files = [f for f in os.listdir("weather_archive") if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("Không tìm thấy file CSV trong thư mục weather_archive")

data_file = os.path.join("weather_archive", csv_files[0])
df = pd.read_csv(data_file)

# Kiểm tra và làm sạch dữ liệu
print("Đang xử lý dữ liệu...")
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna(subset=['Temp_Max', 'Temp_Min', 'Weather_Code'])
df['Weather_Code'] = df['Weather_Code'].astype(int)

# Định nghĩa các mã thời tiết hợp lệ và mô tả
weather_code_descriptions = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Drizzle: Light intensity",
    53: "Drizzle: Moderate intensity",
    55: "Drizzle: Dense intensity",
    56: "Freezing drizzle: Light",
    57: "Freezing drizzle: Dense",
    61: "Rain: Slight",
    63: "Rain: Moderate",
    65: "Rain: Heavy",
    66: "Freezing rain: Light",
    67: "Freezing rain: Heavy",
    71: "Snow fall: Slight",
    73: "Snow fall: Moderate",
    75: "Snow fall: Heavy",
    77: "Snow grains",
    80: "Rain showers: Slight",
    81: "Rain showers: Moderate",
    82: "Rain showers: Violent",
    85: "Snow showers: Slight",
    86: "Snow showers: Heavy",
    95: "Thunderstorm: Slight or moderate",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

# Thêm các đặc trưng thời gian
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfYear'] = df['Date'].dt.dayofyear
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Mã hóa tỉnh thành
le_province = LabelEncoder()
df['Province_Code'] = le_province.fit_transform(df['Province'])

# Tạo thư mục lưu kết quả dự đoán
os.makedirs('xgboost_predictions', exist_ok=True)

# Dự đoán 7 ngày tiếp theo
last_date = df['Date'].max()
future_dates = [last_date + timedelta(days=i+1) for i in range(7)]

# Tạo DataFrame để lưu kết quả dự đoán
prediction_results = pd.DataFrame()

# Lấy danh sách các tỉnh thành
provinces = df['Province'].unique()

print(f"Dự báo thời tiết cho {len(provinces)} tỉnh thành...")

for province_name in provinces:
    print(f"Đang xử lý cho tỉnh: {province_name}")
    
    # Lọc dữ liệu theo tỉnh
    province_data = df[df['Province'] == province_name].sort_values('Date')
    
    # Kiểm tra xem có đủ dữ liệu không
    if len(province_data) < 14:  # Cần ít nhất 14 ngày để có đủ dữ liệu huấn luyện
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu")
        continue
    
    # Tạo các đặc trưng với độ trễ (lag features)
    # Tạo lag features cho nhiệt độ cao, thấp và mã thời tiết
    for lag in range(1, 8):  # Tạo lag từ 1 đến 7 ngày
        province_data[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].shift(lag)
        province_data[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].shift(lag)
        province_data[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].shift(lag)
    
    # Tạo đặc trưng trung bình động (rolling mean)
    for window in [3, 5, 7]:
        province_data[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].rolling(window=window).mean()
        province_data[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].rolling(window=window).mean()
        province_data[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].rolling(window=window).mean()
    
    # Loại bỏ các hàng có giá trị NaN sau khi tạo đặc trưng
    province_data = province_data.dropna()
    
    if len(province_data) < 10:  # Kiểm tra lại sau khi đã tạo đặc trưng
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu sau khi tạo đặc trưng")
        continue
    
    # Chia tập dữ liệu
    train_data = province_data.copy()
    
    # Tạo các đặc trưng cho dự đoán
    feature_cols = [col for col in train_data.columns if col.startswith(('Temp_Max_Lag', 'Temp_Min_Lag', 
                                                                        'Weather_Code_Lag', 'Temp_Max_Roll', 
                                                                        'Temp_Min_Roll', 'Weather_Code_Roll'))]
    feature_cols += ['Month', 'Day', 'DayOfYear', 'DayOfWeek']
    
    X_train = train_data[feature_cols]
    y_temp_max = train_data['Temp_Max']
    y_temp_min = train_data['Temp_Min']
    y_weather_code = train_data['Weather_Code']
    
    # Kiểm tra mã thời tiết có trong dữ liệu của tỉnh này
    unique_weather_codes = y_weather_code.unique()
    print(f"  Mã thời tiết trong dữ liệu của {province_name}: {unique_weather_codes}")
    
    # Huấn luyện mô hình XGBoost cho nhiệt độ cao
    model_temp_max = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )
    model_temp_max.fit(X_train, y_temp_max)
    
    # Huấn luyện mô hình XGBoost cho nhiệt độ thấp
    model_temp_min = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )
    model_temp_min.fit(X_train, y_temp_min)
    
    # Huấn luyện mô hình XGBoost cho mã thời tiết (cũng sử dụng hồi quy)
    model_weather_code = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )
    model_weather_code.fit(X_train, y_weather_code)
    
    # Tìm mã thời tiết gần nhất cho các dự đoán
    def find_nearest_weather_code(pred_code, valid_codes=None):
        if valid_codes is None:
            valid_codes = list(weather_code_descriptions.keys())
        
        valid_codes = np.array(valid_codes)
        idx = (np.abs(valid_codes - pred_code)).argmin()
        return valid_codes[idx]
    
    # Dự đoán cho 7 ngày tiếp theo
    future_predictions = []
    
    # Lấy dữ liệu gần nhất để dự đoán ngày đầu tiên
    last_row = province_data.iloc[-1].copy()
    
    for i in range(7):
        # Chuẩn bị dữ liệu cho ngày dự đoán
        future_date = future_dates[i]
        predict_row = pd.Series(dtype='float64')
        
        # Thêm đặc trưng thời gian
        predict_row['Month'] = future_date.month
        predict_row['Day'] = future_date.day
        predict_row['DayOfYear'] = future_date.dayofyear
        predict_row['DayOfWeek'] = future_date.dayofweek
        
        # Cập nhật lag features dựa trên dự đoán trước đó
        if i == 0:
            # Ngày đầu tiên: sử dụng dữ liệu thực tế
            for lag in range(1, 8):
                if lag <= len(province_data):
                    predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].iloc[-lag]
                    predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].iloc[-lag]
                    predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].iloc[-lag]
                else:
                    # Nếu không đủ dữ liệu, sử dụng giá trị trung bình
                    predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].mean()
                    predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].mean()
                    predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].mean()
            
            # Tính rolling mean dựa trên dữ liệu thực tế
            for window in [3, 5, 7]:
                if window <= len(province_data):
                    predict_row[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].iloc[-window:].mean()
                    predict_row[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].iloc[-window:].mean()
                    predict_row[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].iloc[-window:].mean()
                else:
                    predict_row[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].mean()
                    predict_row[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].mean()
                    predict_row[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].mean()
        else:
            # Từ ngày thứ 2 trở đi: cập nhật lag từ dự đoán trước đó
            for lag in range(1, 8):
                if lag <= i:
                    # Lấy từ dự đoán trước đó
                    predict_row[f'Temp_Max_Lag{lag}'] = future_predictions[i-lag]['Temp_Max']
                    predict_row[f'Temp_Min_Lag{lag}'] = future_predictions[i-lag]['Temp_Min']
                    predict_row[f'Weather_Code_Lag{lag}'] = future_predictions[i-lag]['Weather_Code']
                else:
                    # Lấy từ dữ liệu thực tế
                    idx = lag - i - 1
                    if idx < len(province_data):
                        predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].iloc[-(lag-i)]
                        predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].iloc[-(lag-i)]
                        predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].iloc[-(lag-i)]
                    else:
                        predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].mean()
                        predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].mean()
                        predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].mean()
            
            # Cập nhật rolling mean từ dự đoán trước đó
            for window in [3, 5, 7]:
                if window <= i + 1:
                    # Có đủ dự đoán trước đó + dữ liệu thực tế
                    temp_max_vals = []
                    temp_min_vals = []
                    weather_code_vals = []
                    
                    for w in range(window):
                        if w < i:
                            temp_max_vals.append(future_predictions[i-w-1]['Temp_Max'])
                            temp_min_vals.append(future_predictions[i-w-1]['Temp_Min'])
                            weather_code_vals.append(future_predictions[i-w-1]['Weather_Code'])
                        else:
                            idx = w - i
                            if idx < len(province_data):
                                temp_max_vals.append(province_data['Temp_Max'].iloc[-idx-1])
                                temp_min_vals.append(province_data['Temp_Min'].iloc[-idx-1])
                                weather_code_vals.append(province_data['Weather_Code'].iloc[-idx-1])
                            else:
                                temp_max_vals.append(province_data['Temp_Max'].mean())
                                temp_min_vals.append(province_data['Temp_Min'].mean())
                                weather_code_vals.append(province_data['Weather_Code'].mean())
                    
                    predict_row[f'Temp_Max_Roll{window}'] = np.mean(temp_max_vals)
                    predict_row[f'Temp_Min_Roll{window}'] = np.mean(temp_min_vals)
                    predict_row[f'Weather_Code_Roll{window}'] = np.mean(weather_code_vals)
                else:
                    # Không đủ dữ liệu, sử dụng trung bình
                    predict_row[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].mean()
                    predict_row[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].mean()
                    predict_row[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].mean()
        
        # Chuyển đổi Series thành DataFrame cho dự đoán
        predict_df = pd.DataFrame([predict_row])
        
        # Đảm bảo tất cả các đặc trưng đều có
        for col in feature_cols:
            if col not in predict_df.columns:
                predict_df[col] = 0
        
        # Chỉ giữ lại các đặc trưng trong mô hình
        predict_df = predict_df[feature_cols]
        
        # Dự đoán
        temp_max_pred = model_temp_max.predict(predict_df)[0]
        temp_min_pred = model_temp_min.predict(predict_df)[0]
        weather_code_pred_raw = model_weather_code.predict(predict_df)[0]
        
        # Chuyển đổi dự đoán mã thời tiết về mã thực tế gần nhất
        weather_code_pred = find_nearest_weather_code(weather_code_pred_raw, unique_weather_codes)
        
        # Lấy mô tả thời tiết dựa trên mã
        weather_description = weather_code_descriptions.get(weather_code_pred, "Unknown")
        
        # Thêm vào danh sách dự đoán
        future_predictions.append({
            'Date': future_date,
            'Temp_Max': temp_max_pred,
            'Temp_Min': temp_min_pred,
            'Weather_Code': weather_code_pred,
            'Weather_Description': weather_description
        })
    
    # Chuyển danh sách dự đoán thành DataFrame
    province_predictions = pd.DataFrame(future_predictions)
    province_predictions['Province'] = province_name
    
    # Thêm vào kết quả tổng thể
    prediction_results = pd.concat([prediction_results, province_predictions])
    
    print(f"  ✓ Đã dự báo cho tỉnh {province_name}")

# Xếp lại và lưu kết quả dự đoán (chỉ lưu file này)
prediction_results = prediction_results[['Province', 'Date', 'Temp_Max', 'Temp_Min', 'Weather_Code', 'Weather_Description']]
prediction_results.to_csv('xgboost_predictions/weather_forecast_7days.csv', index=False)

print(f"\nĐã hoàn thành dự báo thời tiết cho {len(provinces)} tỉnh thành!")
print(f"Kết quả được lưu trong thư mục 'xgboost_predictions'")
print(f"File dự báo: 'xgboost_predictions/weather_forecast_7days.csv'")

Đang đọc dữ liệu...
Đang xử lý dữ liệu...
Dự báo thời tiết cho 63 tỉnh thành...
Đang xử lý cho tỉnh: An Giang
  Mã thời tiết trong dữ liệu của An Giang: [63 61 55 51 65  3 53]
  ✓ Đã dự báo cho tỉnh An Giang
Đang xử lý cho tỉnh: Bà Rịa - Vũng Tàu
  Mã thời tiết trong dữ liệu của Bà Rịa - Vũng Tàu: [51  3 53 55 63]
  ✓ Đã dự báo cho tỉnh Bà Rịa - Vũng Tàu
Đang xử lý cho tỉnh: Bạc Liêu
  Mã thời tiết trong dữ liệu của Bạc Liêu: [63 55 51 65  3 53 61]
  ✓ Đã dự báo cho tỉnh Bạc Liêu
Đang xử lý cho tỉnh: Bắc Giang
  Mã thời tiết trong dữ liệu của Bắc Giang: [61  3 51 53 65 63  0]
  ✓ Đã dự báo cho tỉnh Bắc Giang
Đang xử lý cho tỉnh: Bắc Kạn
  Mã thời tiết trong dữ liệu của Bắc Kạn: [51  3 53 65 61 63]
  ✓ Đã dự báo cho tỉnh Bắc Kạn
Đang xử lý cho tỉnh: Bắc Ninh
  Mã thời tiết trong dữ liệu của Bắc Ninh: [61  3 51 65 63 55  1]
  ✓ Đã dự báo cho tỉnh Bắc Ninh
Đang xử lý cho tỉnh: Bến Tre
  Mã thời tiết trong dữ liệu của Bến Tre: [51 55 53  3 61 63]
  ✓ Đã dự báo cho tỉnh Bến Tre
Đang xử lý c